In [1]:
import os
import getpass

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain.vectorstores.chroma import Chroma

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API')

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [4]:
from get_embedding_function import get_embedding_function
CHROMA_PATH = "chroma"

def query_rag(query_text):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text

def query_rag_tool(query_text: str) -> str:
    return query_rag(query_text)

In [5]:
ddg_search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Use esta ferramenta para buscar informações recentes e responda ao usuário com os resultados encontrados. É importante que a resposta contenha o que foi recuperado da pesquisa."
    ),
    Tool(
        name="Query RAG",
        func=query_rag_tool,
        description="A perguntas relacionadas ao turismo da cidade de Natal no Rio Grande do Norte. Se as informações não forem suficientes no Query RAG para responder ao usuário utilize a ferramenta do DuckDuckGo Search"
    )
]

In [6]:
agent_prompt = hub.pull("tales/teste")

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'teste', 'lc_hub_commit_hash': 'a1e1087ebc41da363133b3624e8466fc4a2d480c7ce1157e303b4b0c5e3d7c3d'} template='Seu nome é "Sofia". Você é uma assistente pessoal de inteligência artificial de classe mundial. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações e resolver problemas dos usuários. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática).\n\n\xa0 \xa0 "Sofia" foi projetado para ajudar com uma ampla gama de tarefas, desde responder perguntas simples até fornecer explicações aprofundadas e discussões sobre diversos tópicos. Como um modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas entradas que recebe, permitindo que se envolva em conversas naturais e forneça respostas coerentes e relevantes ao tema discutido.\n\n\xa0 \xa0 "Sofia"

In [8]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_22668\1499110810.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [10]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [11]:
response = agent_executor.invoke({"input": """
                                  Você poderia me ajudar a planejar a minha viagem a cidade de Natal? Eu vou passar três dias entre a sexta e o domingo.
                                  Eu gostaria de visitar as praias mais famosas e comer em restaurantes com uma boa comida regional que não sejam muito caros.
                                  """})

print(response["output"])




> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Planejar viagem de três dias a Natal (RN) incluindo praias famosas e restaurantes regionais baratos.

C:\Users\Tales\AppData\Local\Temp\ipykernel_22668\2325673929.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


8 DIAS (7 NOITES) 
6º Dia Você descobre por que o Rio 
Grande do Norte é a terra do camarão na 
Barra do Cunhaú, grande produtora do 
crustáceo e local de praias desertas.
7º Dia Feche sua viagem na Baía Formosa, 
onde você pode fazer um passeio opcional 
de bugue até a divisa com a Paraíba, 
parando na Lagoa da Coca-Cola.
8º Dia Hora de voltar para casa.
DICAS DE ROTEIROS
Os passeios destacados na sugestão abaixo estão nas páginas 10 a 13
4 DIAS (3 NOITES)
1º Dia: Chegada à cidade. Almoce no 
Restaurante Farofa d’Água, em Ponta 
Negra, para se encantar logo de cara 
com a vista do Morro do Careca.
2º Dia: Faça o Passeio Panorâmico 
para descobrir o melhor de Natal 
e curtir a Praia de Porto Mirim, 
onde você pode fazer um passeio 
opcional de bugue, com brincadeiras

---

PRAIA DA PIPA
Você passa o dia no vilarejo, 
que fica a 85 km de Natal e tem 
praias cercadas por falésias e 
vegetação exuberante. Lá pode 
tomar banho de mar, aproveitar 
o agito das barracas, almoçar 
em um restau

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: melhores restaurantes baratos comida regional Natal RNAs melhores dicas de onde comer em Natal - RN! Recomendamos desde restaurantes clássicos e sofisticados até os alternativos e mais baratos =) ... A comida é regional, tanto do sertão quanto da costa. Então tem muito camarão, frutos do mar, carne de sol, queijo coalho, mandioca - dá para comer um pouco de tudo. ... Restaurante Beira-Mar ... Uma ótima opção de churrascaria dentre os melhores restaurantes em Natal é o Sal e Brasa. Além de uma enorme variedade de cortes de carne, com mais de 25 opções diferentes no rodízio, o restaurante também conta com o buffet, além de servir massas, sobremesas, frutos-do-mar e até mesmo comida japonesa, com sushis e sashimis. O Sal e ... Estou em busca de restaurantes com preços baixos em Natal. @NayaneGoncalves, considere aqui algumas opções de restaurantes, que fui com preços acessíveis em Natal que você pod

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Para uma viagem de três dias a Natal entre sexta e domingo, focando em praias famosas e restaurantes regionais baratos, sugiro o seguinte roteiro:

**Dia 1 (Sexta):**

* **Manhã/Tarde:** Chegue em Natal e vá direto para Ponta Negra. Visite o Morro do Careca (apenas para fotos, a subida é proibida), relaxe na praia e aproveite o ambiente.
* **Noite:** Jante em um restaurante mais econômico em Ponta Negra.  A pesquisa indica que encontrar opções de comida regional barata pode exigir um pouco mais de procura, mas existem opções.  Procure por restaurantes locais e menos turísticos para melhores preços.  Considere pedir pratos com camarão, frutos do mar ou outras especialidades regionais.

**Dia 2 (Sábado):**

* **Manhã/Tarde:** Explore outras praias famosas.  Considere ir a Praia de Pirangi do Norte (conhecida pelo maior cajueiro do mundo) ou a Praia da Redinha (mais próxima do centro e com o Aquário de Natal).  O transporte p

In [12]:
print(response["chat_history"])